In [51]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Hard Baseline (with game data / NOT nba_records) : All-Time**



*   Import game_data


In [195]:
import pandas as pd
game_results = pd.read_csv('/content/drive/Shareddrives/CIS530/data/games_with2_team_tweets.csv', index_col=0)
teams_home = pd.read_csv('/content/drive/Shareddrives/CIS530/data/teams.csv')
teams_away = pd.read_csv('/content/drive/Shareddrives/CIS530/data/teams.csv')
all_games = pd.read_csv('/content/drive/Shareddrives/CIS530/data/games.csv')

Clean Data
*   Keep GAME_DATE_EST, HOME_TEAM_ID, VISITOR_TEAM_ID, HOME_TEAM_WINS
*   Join with teams.csv for Team Names



In [196]:
all_games['GAME_DATE_EST'] = pd.to_datetime(all_games['GAME_DATE_EST'])

In [197]:
game_results = game_results[['GAME_DATE_EST', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'HOME_TEAM_WINS']]
game_results['GAME_DATE_EST'] = pd.to_datetime(game_results['GAME_DATE_EST'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [198]:
teams_home = teams_home.rename(columns={'TEAM_ID': 'HOME_TEAM_ID'})
teams_away = teams_away.rename(columns={'TEAM_ID': 'VISITOR_TEAM_ID'})
teams_home = teams_home[['HOME_TEAM_ID', 'NICKNAME']]
teams_away = teams_away[['VISITOR_TEAM_ID', 'NICKNAME']]

In [158]:
# game_results_final = game_results.merge(teams_home, how='left', on=['HOME_TEAM_ID'])
# game_results_final = game_results_final.rename(columns={'NICKNAME': 'HOME_NAME'})

# game_results_final = game_results_final.merge(teams_away, how='left', on=['VISITOR_TEAM_ID'])
# game_results_final = game_results_final.rename(columns={'NICKNAME':'AWAY_NAME'})

# game_results_final = game_results_final[['GAME_DATE_EST', 'HOME_NAME', 'AWAY_NAME', 'HOME_TEAM_WINS']]
# game_results_final['AWAY_TEAM_WINS'] = [1 if home_win == 0 else 0 for home_win in game_results_final['HOME_TEAM_WINS']]

In [199]:
game_results = game_results.reset_index()

In [200]:
game_results.head(5)

,index,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS
0,3,2021-11-17,1610612751,1610612739,1
1,11,2021-11-16,1610612751,1610612744,0
2,13,2021-11-16,1610612746,1610612759,1
3,14,2021-11-15,1610612739,1610612738,0
4,20,2021-11-15,1610612763,1610612745,1


In [201]:
all_games = all_games.dropna()

In [202]:
all_games.head(5)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-11-17,22100213,Final,1610612766,1610612764,2021,1610612766,97.0,0.438,0.500,0.313,30.0,59.0,1610612764,87.0,0.367,0.813,0.190,23.0,48.0,1
1,2021-11-17,22100214,Final,1610612765,1610612754,2021,1610612765,97.0,0.425,0.750,0.286,16.0,42.0,1610612754,89.0,0.418,0.737,0.243,14.0,43.0,1
2,2021-11-17,22100215,Final,1610612737,1610612738,2021,1610612737,110.0,0.506,0.833,0.351,28.0,40.0,1610612738,99.0,0.440,0.824,0.268,24.0,42.0,1
3,2021-11-17,22100216,Final,1610612751,1610612739,2021,1610612751,109.0,0.458,0.840,0.375,29.0,47.0,1610612739,99.0,0.393,0.857,0.250,20.0,50.0,1
4,2021-11-17,22100217,Final,1610612748,1610612740,2021,1610612748,113.0,0.483,0.824,0.375,29.0,39.0,1610612740,98.0,0.440,0.786,0.286,18.0,38.0,1




*   Calculate Win/Loss before each game for both teams
 *  For each game, calculate the number of losses and wins for each team previous to the date of that game


In [203]:
def get_strong_baseline_home(row):
    home_team_games_playing_home = all_games[(all_games["HOME_TEAM_ID"] == row["HOME_TEAM_ID"])
                                     &(all_games["GAME_DATE_EST"] < row["GAME_DATE_EST"])]
    home_team_games_playing_away = all_games[(all_games["VISITOR_TEAM_ID"] == row["HOME_TEAM_ID"])
                                     &(all_games["GAME_DATE_EST"] < row["GAME_DATE_EST"])]
    total_home_team_games = home_team_games_playing_home.shape[0] + home_team_games_playing_away.shape[0]
 
    home_team_wins_playing_home = home_team_games_playing_home[home_team_games_playing_home["HOME_TEAM_WINS"]==1].shape[0]
    home_team_wins_playing_away = home_team_games_playing_away[home_team_games_playing_away["HOME_TEAM_WINS"]==0].shape[0]
    home_team_wins = home_team_wins_playing_home + home_team_wins_playing_away
 
    home_team_losses_playing_home = home_team_games_playing_home[home_team_games_playing_home["HOME_TEAM_WINS"]==0].shape[0]
    home_team_losses_playing_away = home_team_games_playing_away[home_team_games_playing_away["HOME_TEAM_WINS"]==1].shape[0]
    home_team_losses = home_team_losses_playing_home + home_team_losses_playing_away
 
    return home_team_wins/home_team_losses


 
home_win_loss = game_results.apply(get_strong_baseline_home, axis=1)

4035

In [204]:
def get_strong_baseline_away(row):
    away_team_games_playing_away = all_games[(all_games["VISITOR_TEAM_ID"] == row["VISITOR_TEAM_ID"])
                                     &(all_games["GAME_DATE_EST"] < row["GAME_DATE_EST"])]
    away_team_games_playing_home = all_games[(all_games["HOME_TEAM_ID"] == row["VISITOR_TEAM_ID"])
                                     &(all_games["GAME_DATE_EST"] < row["GAME_DATE_EST"])]
    total_away_team_games = away_team_games_playing_away.shape[0] + away_team_games_playing_home.shape[0]
 
    away_team_wins_playing_away = away_team_games_playing_away[away_team_games_playing_away["HOME_TEAM_WINS"]==0].shape[0]
    away_team_wins_playing_home = away_team_games_playing_home[away_team_games_playing_home["HOME_TEAM_WINS"]==1].shape[0]
    away_team_wins = away_team_wins_playing_away + away_team_wins_playing_home
 
    away_team_losses_playing_away = away_team_games_playing_away[away_team_games_playing_away["HOME_TEAM_WINS"]==1].shape[0]
    away_team_losses_playing_home = away_team_games_playing_home[away_team_games_playing_home["HOME_TEAM_WINS"]==0].shape[0]
    away_team_losses = away_team_losses_playing_away + away_team_losses_playing_home
 
    return away_team_wins/away_team_losses


 
away_win_loss = game_results.apply(get_strong_baseline_away, axis=1)

In [ ]:
print(home_win_loss)
away_win_loss



*   Set "Actual" result
*   Set "Predicted" result (based on highest win/loss)


In [213]:
game_results['HOME_W_L'] = home_win_loss
game_results['AWAY_W_L'] = away_win_loss

In [214]:
game_results.head(5)

,index,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,HOME_W_L,AWAY_W_L
0,3,2021-11-17,1610612751,1610612739,1,0.789760,1.033413
1,11,2021-11-16,1610612751,1610612744,0,0.790622,1.270525
2,13,2021-11-16,1610612746,1610612759,1,1.075871,1.847896
3,14,2021-11-15,1610612739,1610612738,0,1.034648,1.246154
4,20,2021-11-15,1610612763,1610612745,1,0.965394,1.336565


In [219]:
game_results['PRED_HOME_TEAM_WINS'] = game_results.apply(lambda row: 1 if row.HOME_W_L >= row.AWAY_W_L else 0, axis=1)

In [222]:
actual = game_results['HOME_TEAM_WINS']
predicted = game_results['PRED_HOME_TEAM_WINS']

Accuracy: F1 and accuracy

In [228]:
from sklearn.metrics import f1_score, accuracy_score
f1_score = f1_score(actual, predicted)
accuracy_score = accuracy_score(actual, predicted)
print("F1: " + str(f1_score), "Accuracy: " + str(accuracy_score))

F1: 0.5806598407281 Accuracy: 0.5432465923172243


**Hard Baseline (with game data / NOT nba_records) : Per Season**
